# 高度な分析: 予測モデルと地理空間分析

このノートブックでは、分煙対策データを使用してより高度な分析を行います。
- 時系列予測
- 地理空間分析
- 機械学習による分類・予測


In [ ]:
# 高度な分析用ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import folium
from folium.plugins import HeatMap

# 日本語フォント設定
plt.rcParams['font.family'] = ['DejaVu Sans', 'Hiragino Sans', 'Yu Gothic', 'Meiryo', 'Takao', 'IPAexGothic', 'IPAPGothic', 'VL PGothic', 'Noto Sans CJK JP']

# 警告を非表示
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline


## 時系列予測モデル

機械学習を使用して将来の指導件数を予測します。


In [ ]:
# データの読み込みと前処理
df = pd.read_csv('../data/131130_分煙対策_指導と過料処分の活動実績_1924926636818441489.csv')

# 日付関連の特徴量を作成
df['日付'] = pd.to_datetime(df[['年度', '月', '日']])
df['曜日'] = df['日付'].dt.dayofweek
df['月'] = df['日付'].dt.month
df['四半期'] = df['日付'].dt.quarter

# 路上喫煙指導データのみを抽出
smoking_data = df[df['値区分'] == '路上喫煙'].copy()

# 日別の集計データを作成
daily_summary = smoking_data.groupby('日付').agg({
    '値': 'sum',
    '曜日': 'first',
    '月': 'first',
    '四半期': 'first'
}).reset_index()

print(f"予測用データ形状: {daily_summary.shape}")
print(f"期間: {daily_summary['日付'].min()} から {daily_summary['日付'].max()}")
daily_summary.head()
